In [ ]:
#module1.0
#retrieving drug information from KEGG DRUG (https://www.kegg.jp/kegg/drug/)
import urllib.request
import pandas as pd
from bs4 import BeautifulSoup 
import time


def name_clean(string):#removing "\n" and space
    i = 0
    if len(string)>1:
        while string[i]==" " or string[i]=="\n":
            i = i+1
        string_clean = string[i:]
    else:
        string_clean = string
    return string_clean

def efficay_retrive(drug_ID):
    html ="https://www.kegg.jp/entry/{}".format(drug_ID)
    while True:#to prevent HTTP403 error
        try:
           response = urllib.request.urlopen(html)
           break
        except:
           time.sleep(20)
    soup = BeautifulSoup(response)
    cel = list(soup.find_all(["div","span"]))
    ls_text = []#list for get_text
    for i, cel_index in enumerate(cel):
       ls_text.append(cel_index.get_text())
    try:
       efficacy = ls_text[ls_text.index("Efficacy")+1].split(", ")#get text after "Efficacy" element, then convert list format by splitting
    except Exception as e:
       print("exception; "+str(e.args))
       efficacy = []
    print(efficacy)
    return efficacy

j = 1 #index number
i = 1 #page number
max_page = 303 #enter the number of pages
efficacy_len = 1 #the number of column of efficacy
dic={} #the dictionary; key = index number (j), values = drug information
for i in range(max_page):
    html ="https://www.kegg.jp/medicus-bin/search_medicus?structure=off&uid=167687911843111&search_gene=1&display=drug&page={}&from=drug".format(i+1)
    while True:#to prevent HTTP403 error
       try:
          response = urllib.request.urlopen(html)
          break
       except:
          time.sleep(20)

    soup = BeautifulSoup(response)
    tr = soup.find_all("tr")
    for data in tr:
         data1 = data.find_all("td",class_="data1")#retrieve drug_ID, Name
         ls1 = [] #list to store "Entry","Name","Name(TN)",and "Disease"
         for tag in data1:
            tag_clean = name_clean(tag.get_text())
            print(tag_clean)
            ls1.append(tag_clean)
         if len(ls1)>0: #exlude empty data
            efficacy = efficay_retrive(ls1[0])#ls1[0]=drug_ID (Entry)
            efficacy_len = max(efficacy_len,len(efficacy))#update efficacy_len
            ls1.extend(efficacy)
            dic[j]=ls1
            j = j+1
         print("=============\n")
    response.close()


columns = ["Entry",	"Name",	"Name(TN)",	"Disease"]+["efficacy{}".format(i+1) for i in range(efficacy_len)]
print("Information of "+str(len(dic))+" drugs were retrieved")
KEGG_df = pd.DataFrame(list(dic.values()),columns=columns)
KEGG_df.set_index("Entry", inplace=True)
display(KEGG_df)
KEGG_df.to_csv("../Files/Figure2/Table1.csv")